In [14]:
from sympy import symbols, diff, Eq, solve, lambdify

# Define variables
y, lambda_symbol = symbols('y lambda')

# Define b(y)
b_y = 1 / (100 + y) ** 2

# Define Profit function P(y)
P_y = y * b_y

# Find derivative of Profit function and solve for y
P_prime_y = diff(P_y, y)
critical_points = solve(P_prime_y, y)

# Check the critical points found to determine the value of y that maximizes profit
y_max_profit = critical_points[0] if len(critical_points) > 0 else 0

# Define the Lagrangian L = P(y) + λ(ȳ - y)
y_bar = symbols('y_bar')  # ȳ
Lagrangian = P_y + lambda_symbol * (y_bar - y)

# Solve the Lagrangian for given ȳ values
y_bars = [75, 150]
lagrange_multipliers = []

for y_value in y_bars:
    # Take the derivative of Lagrangian with respect to y and λ, and solve
    dL_dy = diff(Lagrangian.subs(y_bar, y_value), y)
    dL_dlambda = diff(Lagrangian.subs(y_bar, y_value), lambda_symbol)
    solutions = solve((dL_dy, dL_dlambda), (y, lambda_symbol))

    print(solutions)    
    
    lagrange_multipliers.append({
        'y_bar': y_value,
        'y': solutions[0][0],
        'lambda': solutions[0][1]
    })

# Print Results
print(f"To maximize profit, buy and sell {y_max_profit.evalf()} units.")
for lagrange_multiplier in lagrange_multipliers:
    print(f"For y_bar = {lagrange_multiplier['y_bar']}, y = {lagrange_multiplier['y'].evalf()}, λ = {lagrange_multiplier['lambda'].evalf()}")



[(75, 1/214375)]
[(150, -1/312500)]
To maximize profit, buy and sell 100.000000000000 units.
For y_bar = 75, y = 75.0000000000000, λ = 0.00000466472303206997
For y_bar = 150, y = 150.000000000000, λ = -0.00000320000000000000
